# Detection with YOLO

In this section, we will begin exploring our detection pipeline using **YOLOv11** as the primary object detector.

1. Relabelling the drowning detection dataset for human detection task
2. A series of **visualizations** to understand how YOLO processes each image — including bounding box predictions, confidence scores, and the role of preprocessing.
3. Experimentation with **hyperparameter tuning** to adjust settings like learning rate, batch size, and optimizer behavior.
4. **Fine-tuning** the YOLO model on the drowning detection dataset to improve accuracy in real-world aquatic scenarios.

This serves as the foundation before integrating it with a secondary classifier for behavior prediction (e.g., swimming, treading water, drowning).


In [1]:
from dotenv import load_dotenv

load_dotenv()

import sys
import os

sys.path.append(os.getenv('SRC_DIR'))

from utils.system import display_system_info

display_system_info(markdown=True)


**Last Updated**: 2025-04-19 13:55:23

**Python Version**: 3.11.11  
**OS**: Windows 10.0.26100  
**Architecture**: 64bit  
**Hostname**: ShenLaptop  
**Processor**: Intel64 Family 6 Model 186 Stepping 3, GenuineIntel  
**RAM Size**: 15.65 GB  
  
        

In [2]:
from ultralytics import YOLO
import shutil

## Relabelling for YOLO Training

In [3]:
yolo_training_data_dir = os.getenv('YOLO_TRAINING_DATA_DIR')
raw_data_dir = os.getenv('RAW_DATA_DIR')
img_dir = os.getenv('IMG_DIR')
label_dir = os.getenv('LABEL_DIR')
train_dir = os.getenv('TRAIN_DIR')
val_dir = os.getenv('VAL_DIR')
yolo_config_dir = os.getenv('YOLO_CONFIG_DIR')

In [ ]:
for split in [train_dir, val_dir]:
    img_src_dir = os.path.join(raw_data_dir, img_dir, split)
    label_src_dir = os.path.join(raw_data_dir, label_dir, split)

    img_dst_dir = os.path.join(yolo_training_data_dir, img_dir, split)
    label_dst_dir = os.path.join(yolo_training_data_dir, label_dir, split)

    os.makedirs(img_dst_dir, exist_ok=True)
    os.makedirs(label_dst_dir, exist_ok=True)

    for filename in os.listdir(img_src_dir):
        if filename.endswith(('.jpg', )):
            shutil.copy2(os.path.join(img_src_dir, filename), os.path.join(img_dst_dir, filename))

            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_src_path = os.path.join(label_src_dir, label_filename)
            label_dst_path = os.path.join(label_dst_dir, label_filename)

            if os.path.exists(label_src_path):
                with open(label_src_path, 'r') as f:
                    lines = f.readlines()
                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        parts[0] = '0'  # change class ID to 0
                        new_lines.append(' '.join(parts))
                with open(label_dst_path, 'w') as f:
                    f.writelines(line + '\n' for line in new_lines)
            else:
                # If no label exists, create an empty label file (optional)
                open(label_dst_path, 'w').close()

## YOLO Fine-Tuning

In [ ]:
yaml_content = (
f"""# Dataset paths
path: {yolo_training_data_dir}
train: {os.path.join(img_dir, train_dir)}
val: {os.path.join(img_dir, val_dir)}

# Classes
nc: 1
names: ['person']
""")

yaml_path = os.path.join(yolo_config_dir, "fine_tuning.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)
print(f"Created data configuration at {yaml_path}")

In [ ]:
import torch

model = YOLO(os.path.join(os.getenv('YOLO_DIR'), 'yolo11n.pt'))

results = model.train(
    data=yaml_path,
    epochs=50,
    patience=10,
    batch=16,
    imgsz=640,
    save=True,
    save_period=1,
    cache=False,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=12,
    project=os.getenv('YOLO_DIR'),
    name='yolo11n-finetuned',
    exist_ok=True,
    optimizer='auto',
    seed=32,
    deterministic=False,
    single_cls=False,
    resume=False,
    fraction=1,
    dropout=0,
    val=True,
    plots=True,
    degrees=10,
    mosaic=1
)